In [1]:
import sys
import datetime as dt
from getpass import getuser

import pandas as pd
from sqlalchemy import text

sys.path.append(r'C:\Users\user\Desktop\github\sotrans_analytics_department')
from source.core.de.serving.datamart.modules.config import DEFAULT_ENGINE

In [2]:
ROOT_FOLDER: str = r'C:\Users\user\Desktop\github\sotrans_analytics_department'
TRADE_FOLDER: str = r'\\192.168.101.228\Trade'

In [3]:
CURRENT_ORDERS_FILE: str = rf'{TRADE_FOLDER}\2025\Orders\Подтвержденные заказы 2025.xlsx'

In [4]:
# Сформировать датафрейм по текущим размещённым заказам;
orders: pd.DataFrame = pd.read_excel(
    io=CURRENT_ORDERS_FILE,
    engine='openpyxl',
    usecols=[1, 7, 17, 20, 26, 27],
    skiprows=1
)

In [5]:
# Переименование столбцов;
orders: pd.DataFrame = (
    orders
    .rename(
        columns={
            'Код 1С': 'sku_id_1c',
            'Заказано кол-во ': 'ordered_cnt',
            'Бэкордер кол-во': 'backorder_cnt',
            'Отгруж кол-во': 'shipped_cnt',
            'Текущий статус': 'order_status',
            'Ожидаемая дата прихода на склад': 'arrival_date'
        }
    )
)

In [6]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1245 entries, 0 to 1244
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   sku_id_1c      839 non-null    object        
 1   ordered_cnt    1241 non-null   float64       
 2   backorder_cnt  1244 non-null   float64       
 3   shipped_cnt    0 non-null      float64       
 4   order_status   359 non-null    object        
 5   arrival_date   176 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 58.5+ KB


In [7]:
# Фильтрация пропусков;
orders: pd.DataFrame = orders.dropna(subset='sku_id_1c')
orders: pd.DataFrame = orders[orders['sku_id_1c'] != '        ']

In [8]:
# Приведение к нижнему регистру;
orders['sku_id_1c'] = orders['sku_id_1c'].str.lower()
orders['order_status'] = orders['order_status'].str.lower()

In [9]:
# Замена пропусков в shipped_cnt;
orders.loc[:, 'shipped_cnt'] = orders['shipped_cnt'].fillna(value=0)

In [10]:
# Замена пропусков в order_status;
orders.loc[:, 'order_status'] = orders['order_status'].fillna(value='_нет данных')

In [11]:
# Замена пропусков в shipped_cnt;
new_arrival_date = pd.Timestamp.today() + pd.DateOffset(days=180)
orders.loc[:, 'arrival_date'] = orders['arrival_date'].fillna(value=new_arrival_date.date())

In [12]:
# Группировака и аггрегация
orders: pd.DataFrame = (
    orders
    .groupby(
        by=[
            'sku_id_1c',
            'order_status',
            'arrival_date'
        ]
    )
    .aggregate(
        {
            'ordered_cnt': 'sum',
            'backorder_cnt': 'sum',
            'shipped_cnt': 'sum'
        }
    )
    .reset_index()
)

In [13]:
orders.loc[:, 'last_update_at'] = pd.Timestamp.today()

In [14]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   sku_id_1c       801 non-null    object        
 1   order_status    801 non-null    object        
 2   arrival_date    801 non-null    datetime64[ns]
 3   ordered_cnt     801 non-null    float64       
 4   backorder_cnt   801 non-null    float64       
 5   shipped_cnt     801 non-null    float64       
 6   last_update_at  801 non-null    datetime64[us]
dtypes: datetime64[ns](1), datetime64[us](1), float64(3), object(2)
memory usage: 43.9+ KB


# Проверка наличия дат поступления < текущей даты

In [15]:
mask = orders[orders['arrival_date'] < dt.datetime.today()].index

In [16]:
if orders[orders['arrival_date'] < dt.datetime.today()].shape[0] > 0:
    print(f'Количество строк с просроченными датами: {orders[orders['arrival_date'] < dt.datetime.today()].shape[0]}')
    mask = orders[orders['arrival_date'] < dt.datetime.today()].index
    orders.loc[mask, ['arrival_date']] = new_arrival_date

Количество строк с просроченными датами: 54


# Дополнительная проверка

In [17]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [18]:
print(*orders['order_status'].unique(), sep='\n')

в производстве
готов, 2 ктк
готов (1 паллет 100*80*120)
готов
_нет данных
заказ готов, 2 ктк
готов, 2*20
готов 9 палл
готов 3 палл
готов 5 палл
готов 15 пал
готов 4 пал
готов 3*20
готов 1*40


In [19]:
orders[orders['arrival_date'] < dt.datetime.today()]

,sku_id_1c,order_status,arrival_date,ordered_cnt,backorder_cnt,shipped_cnt,last_update_at


In [20]:
orders[orders['shipped_cnt'] > 0]

,sku_id_1c,order_status,arrival_date,ordered_cnt,backorder_cnt,shipped_cnt,last_update_at


# SAVE DATA

In [21]:
with DEFAULT_ENGINE.begin() as connection:
    connection.execute(text("TRUNCATE TABLE report.cr_orders;"))
    
    orders.to_sql(
        name='cr_orders',
        con=connection,
        schema='report',
        index=False,
        if_exists='append'
    )